In [1]:
import numpy as np

from kifmm_py import (
    KiFmmMulti,
    LaplaceKernel,
    HelmholtzKernel,
    MultiNodeTree,
    EvalType,
    BlasFieldTranslation,
    FftFieldTranslation,
    SortKind,
lib,
ffi
)

In [2]:
from mpi4py import MPI

world = MPI.COMM_WORLD

In [3]:
np.random.seed(0)

dim = 3
dtype = np.float32

# Set FMM Parameters
expansion_order = np.array([6], np.uint64)  # Single expansion order as using n_crit
n_vec = 1
n_crit = 150
n_sources = 1000
n_targets = 2000
prune_empty = True  # Optionally remove empty leaf boxes, their siblings, and ancestors, from the Tree

# Setup source/target/charge data in Fortran order
sources = np.random.rand(n_sources * dim).astype(dtype)
targets = np.random.rand(n_targets * dim).astype(dtype)
charges = np.random.rand(n_sources * n_vec).astype(dtype)

eval_type = EvalType.Value

# EvalType computes either potentials (EvalType.Value) or potentials + derivatives (EvalType.ValueDeriv)
kernel = LaplaceKernel(dtype, eval_type)

local_depth = 1
global_depth = 1
sort_kind = SortKind.SampleSort
n_samples = 10

tree = MultiNodeTree(world, sources, targets, charges, local_depth, global_depth, sort_kind, prune_empty, n_samples)

In [4]:
field_translation = FftFieldTranslation(kernel, block_size=32)

In [5]:
tree.communicator_c

<cdata 'void *' NULL>

In [6]:
print(ffi.typeof(lib.laplace_fft_f32_mpi_alloc))


<ctype 'struct FmmEvaluatorMPI *(*)(_Bool, uintptr_t *, uintptr_t, _Bool, void *, uintptr_t, void *, uintptr_t, void *, uintptr_t, _Bool, uint64_t, uint64_t, uintptr_t, uint64_t, uintptr_t, void *)'>


In [7]:
fmm = KiFmmMulti(expansion_order, tree, field_translation, timed=True)

HERE SortKind.SampleSort


TypeError: an integer is required